In [1]:
#Load Data
import pandas as pd

df = pd.read_csv('cleaned_suicide_data.csv')
df = df[0:100000]

In [2]:
!pip install gensim
import nltk
from gensim.utils import simple_preprocess
import gensim
from gensim.models import CoherenceModel

In [3]:
import gensim.corpora as corpora

#create joined class corpus
joined_words_old = [str(d_j).split() for d_j in df["lemmatized_text"]]
joined_words = []
for list1 in joined_words_old:
    temp_arr = []
    for words in list1:
        if len(words) > 2:
            temp_arr.append(words)
    joined_words.append(temp_arr)
id2word_joined = corpora.Dictionary(joined_words)
texts_joined = joined_words
joined_corpus = [id2word_joined.doc2bow(text3) for text3 in texts_joined]
print(joined_corpus[:1][0][:30])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2)]


In [4]:
#joined class lda model
from pprint import pprint

# number of topics
num_topics = 10
# Build LDA model
lda_model_j = gensim.models.LdaMulticore(corpus=joined_corpus, id2word=id2word_joined, num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model_j.print_topics())
doc_lda_j = lda_model_j[joined_corpus]

[(0,
  '0.021*"get" + 0.014*"like" + 0.010*"make" + 0.010*"feel" + 0.010*"say" + '
  '0.009*"want" + 0.009*"one" + 0.008*"really" + 0.008*"fuck" + 0.007*"life"'),
 (1,
  '0.025*"want" + 0.015*"get" + 0.014*"feel" + 0.012*"like" + 0.011*"people" + '
  '0.010*"know" + 0.009*"life" + 0.009*"think" + 0.008*"well" + 0.008*"make"'),
 (2,
  '0.015*"get" + 0.013*"like" + 0.012*"day" + 0.011*"life" + 0.011*"know" + '
  '0.011*"would" + 0.009*"feel" + 0.009*"think" + 0.009*"want" + 0.009*"make"'),
 (3,
  '0.016*"get" + 0.012*"know" + 0.011*"want" + 0.010*"say" + 0.010*"one" + '
  '0.010*"year" + 0.009*"make" + 0.009*"friend" + 0.009*"feel" + 0.008*"good"'),
 (4,
  '0.016*"know" + 0.015*"life" + 0.014*"want" + 0.012*"fuck" + 0.010*"feel" + '
  '0.010*"get" + 0.008*"would" + 0.008*"like" + 0.007*"amp" + 0.007*"paul"'),
 (5,
  '0.014*"know" + 0.013*"people" + 0.011*"want" + 0.010*"get" + 0.009*"would" '
  '+ 0.009*"like" + 0.008*"say" + 0.007*"think" + 0.007*"make" + '
  '0.007*"friend"'),
 (6,
  '

In [5]:
#cohernece score for joined class

coherence_model_lda_j = CoherenceModel(

   model=lda_model_j, texts=texts_joined, dictionary=id2word_joined, coherence='c_v')

coherence_lda_j = coherence_model_lda_j.get_coherence()

print('\nCoherence Score Joined Class: ', coherence_lda_j)


Coherence Score Joined Class:  0.3777887998128312


In [6]:
!pip install pyLDAvis
import pyLDAvis.gensim

In [7]:
#suicide class unigram visualisation
topic_data =  pyLDAvis.gensim.prepare(lda_model_j, joined_corpus, id2word_joined, mds = 'pcoa')
pyLDAvis.display(topic_data)

In [8]:
print("Done")

Done


In [9]:
top_words_per_topic_unigram_100k = []
for t in range(lda_model_j.num_topics):
    top_words_per_topic_unigram_100k.extend([(t, ) + x for x in lda_model_j.show_topic(t, topn = 10)])

pd.DataFrame(top_words_per_topic_unigram_100k, columns=['Topic', 'Word', 'P']).to_csv("top_words_unigram_100k.csv")